# Reports

In [91]:
import os
import re
from datetime import datetime
current_datetime = datetime.now().strftime('%Y%m%d.%H%M')

import numpy as np
import pandas as pd

# for the notebook rendering 
from IPython.display import display, HTML, FileLink, Markdown
from IPython.display import FileLink as FL
from IPython.display import Markdown as MD


DIR = os.getcwd()
print(f'{DIR=}')

RDIR = os.path.join(DIR,'reports')
print(f'{RDIR=}')

DIR='c:\\Users\\JGarza\\GitHub\\wifi_monitor'
RDIR='c:\\Users\\JGarza\\GitHub\\wifi_monitor\\reports'


In [92]:
template_path = os.path.join(DIR,'template.html')

In [93]:
def break_down_buckets(idf,column,buckets,message='',nan_value=-1):
    """
    breaks a column down into buckets/bins
    """
    idf = idf.fillna(nan_value)

    # print('',message,'\ncolumn: ',column, '\nbuckets: ', buckets)
    
    idf = pd.DataFrame(idf[column])
    idf['bucket'] = pd.cut(idf[column], bins=buckets)
    idf = idf.groupby(by='bucket',observed=False).count()
    idf['percent'] = (idf[column]/idf[column].sum())*100
    idf['percent'] = idf['percent'].round(2)
    # display(idf)

    return idf

#test
# break_down_buckets(df,'AGE_YRS',[-1,0,15,25,35,45,55,65,75,85,500])

In [94]:
def bar(num,denom=100.0,length=30,fillchar='█',emptychar='░'):
    fillnum = ((int)( (num/denom) * length))
    return '' + ( fillnum * fillchar ).ljust(length,emptychar)  + '' # + f" {(num/denom)*100.0:.2f}%     " 

In [95]:
# YYYYMMDDHHmm
DateFilter = None
DateFilter = 202501100000 # new firmware r44715
DateFilter =   202508312000

In [96]:
data = {}

for i in os.listdir(RDIR):
    # print(i)
    try:

        text = ''
        with open(os.path.join(RDIR, i), 'r') as f:
            text = f.read()
        # print(text)

        dt = re.findall('Datetime_alt: .*\n',text)[0]
        dt = dt.replace('Datetime_alt: ','').replace('\\n','')
        dt = int(dt)
            
        if DateFilter != None:
            if dt < DateFilter:
                #move on to next
                # print('exclude',dt,DateFilter)
                continue
            else:
                print('include',dt,DateFilter)

        wifi = re.findall('^SSID:.*\n',text)[0]
        wifi = wifi.replace('\n', '').replace('SSID: ','')
        # wifi = wifi + " " + getfirmware(wifi,dt)

        pings = re.findall('.*time=.*\n',text)
        pings = [ re.sub('.*time=','',p) for p in pings]
        pings = [ re.sub('ms.*\n','',p) for p in pings]
        pings = [ float(p.strip()) for p in pings]
        # print(*pings,sep='\n')


        signalstrength =  None
        try:
            signalstrength = re.findall('Signal_Strength:.*\n',text)[0]
            signalstrength = signalstrength.replace('\n', '').replace('Signal_Strength: ','').replace('%','')
        except:
            pass

        timeouts = re.findall('Request timed out.\n',text)

        trycount = re.findall('(Request|Reply).*\n',text)

        if wifi in data.keys():
            pass
        else:
            data[wifi] = {}
            data[wifi]['pings'] = []
            data[wifi]['timeouts'] = []
            data[wifi]['trycount'] = []
            data[wifi]['signalstrength'] = []
        
        data[wifi]['pings'].extend(pings)
        data[wifi]['timeouts'].extend([len(timeouts)])
        data[wifi]['trycount'].extend([len(trycount)])

        if signalstrength != None:
            data[wifi]['signalstrength'].append(int(signalstrength))



    except Exception as e:
        pass
        print(i,e)


Error404NH_202412292039.txt list index out of range
Error404_202412292041.txt list index out of range
include 202508312130 202508312000
Error404_EXT_202412292045.txt list index out of range
include 202508312132 202508312000
include 202508312135 202508312000
include 202508312137 202508312000


In [97]:
print(data)

{'Error 404': {'pings': [11.5, 11.1, 7.88, 11.1, 8.56, 10.9, 9.97, 9.92, 9.39, 11.6, 7.59, 35.9, 66.0, 20.0, 13.3, 10.3, 9.98, 8.75, 12.3, 9.44, 8.83, 10.0, 8.9, 11.8, 9.96, 10.9, 9.05, 9.59, 8.34, 11.9, 9.58, 10.3, 8.26, 7.93, 10.8, 10.6, 9.28, 10.8, 8.78, 11.4, 9.59, 11.1, 9.6, 13.5, 49.3, 19.7, 11.0, 12.0, 8.12, 9.11, 9.08, 12.0, 8.26, 11.7, 11.3, 11.2, 7.75, 9.74, 7.97, 9.52, 10.5, 11.6, 9.09, 9.04, 9.15, 11.1, 8.65, 12.1, 8.03, 10.1, 8.83, 12.1, 9.98, 10.0, 8.8, 23.7, 43.0, 12.8, 10.9, 10.7, 10.1, 10.0, 8.87, 10.2, 9.79, 20.2, 11.7, 10.0, 10.0, 22.2, 11.6, 8.37, 11.6, 12.5, 11.0, 8.15, 10.0, 12.7, 14.3, 12.2], 'timeouts': [0], 'trycount': [0], 'signalstrength': [46]}, 'Error 404_EXT': {'pings': [15.8, 14.0, 12.7, 13.3, 12.4, 13.0, 13.5, 10.9, 11.2, 12.8, 13.9, 13.0, 12.7, 14.4, 20.4, 13.7, 14.1, 14.7, 12.0, 12.1, 19.1, 15.0, 11.5, 11.9, 14.4, 15.3, 15.1, 18.1, 11.5, 13.7, 15.3, 11.4, 17.2, 12.5, 15.8, 13.9, 13.5, 11.9, 12.2, 16.6, 13.4, 13.2, 10.4, 14.7, 12.0, 16.9, 13.7, 13.1, 11

In [98]:
segment = """
<h2 class=""><i class="nf nf-md-wifi"></i>Wifi: {%segment_header%}</h2>
<h3 class="">Signal Strength</h2>

    <div class="progress bg-dark bar-boarder SSbar" style="height: 25px;">
        {%signalstrength%}
    </div>

<br />
<h3 class="">Ping Table</h2>
{%ping_table%}

<div class="divider"></div>
"""

In [99]:
template = ''
with open(template_path, 'r', encoding='utf8') as f:
    template = f.read()

template = template.replace('{%report_header%}',str(current_datetime))

segments = ''
for k in data.keys():
    # print(f"{k=}")
    
    ns = segment

    ns = ns.replace('{%segment_header%}',k)

    ss = ""
    for i in data[k]['signalstrength']:
        # print(i)
        ss += f"""<div class="signalstrength-point rounded rounded-circle" style="left: {i}%;"></div>"""

    ns = ns.replace('{%signalstrength%}',ss)

    # ssmin = min(data[k]["signalstrength"])
    # ssmean = sum(data[k]["signalstrength"])/len(data[k]["signalstrength"])
    # ssmax = max(data[k]["signalstrength"])

    # ns = ns.replace('{%zeromin%}',f'{ssmin:.2f}')
    # ns = ns.replace('{%minmean%}',f'{ssmean-ssmin-2:.2f}')
    # ns = ns.replace('{%meanmean%}',f'{4:.2f}')
    # ns = ns.replace('{%meanmax%}',f'{ssmax-ssmean-2:.2f}')

    # ns = ns.replace('{%ssmin%}',f'{ssmin:.0f}')
    # ns = ns.replace('{%ssmean%}',f'{ssmean:.0f}')
    # ns = ns.replace('{%ssmax%}',f'{ssmax:.0f}')

    df_pings = pd.DataFrame(data[k]['pings'])
    try:
        bdb = break_down_buckets(df_pings,0,[0,5,10,15,20,30,40,50,500])
        # bdb = break_down_buckets(df_pings,0,range(0,100,5))
        bdb['bar'] = bdb.percent.apply(bar)
        bdb = bdb.reset_index()
    except Exception as e:
        # print(f"k")
        print(data[k])
        print(str(e))
        # print(f'Error: {e}')
        continue

    # ns = ns.replace('{%ping_table%}',bdb.to_html(index=False,justify='left'))
    # ns = ns.replace('\"dataframe\"','\"table table-dark table-striped\"')

    failed_pings = sum(data[k]['timeouts'])
    failed_percent = "0"
    try:
        failed_percent = f"{failed_pings/sum(data[k]['trycount']):.0f}"
    except:
        pass

    tbl = ''
    tbl += '<table class="table table-dark table-stripedx">\n'
    tbl += """
    <thead>
        <tr style="text-align: left;">
        <th>bucket</th>
        <th></th>
        </tr>
    </thead>
    <tbody>
    """
    for i,row in bdb.iterrows():
        # print(i)
        # bucket = row['bucket']
        bucket = re.sub(r'[\(\]]', '', str(row['bucket']))
        bucket = re.sub(r',', '-', bucket)
        bucket = re.sub(r' ', '', bucket)
        tbl += f"""
            <tr>
            <td class="col-1">{bucket}</td>
            <td class="col-12">
                <div class="progress bar-boarder bg-dark">
                <div class="progress-bar overflow-visible bar-ping rounded" style="width: {row['percent']}%; height:25px">
                    <span class="badge text-start">
                    {row[0]} | {row['percent']}%
                    </span>
                </div>
                </div>
            </td>
            </tr>
        """
        
    tbl += f"""
        <tr>
        <td class="col-1 text-danger">Failed</td>
        <td class="col-12">
            <div class="progress bar-boarder-danger bg-dark">
            <div class="progress-bar overflow-visible bg-danger  rounded" style="width: {failed_percent}%; height:25px">
                <span class="badge-failed text-start">
                {failed_pings} | {failed_percent}%
                </span>
            </div>
            </div>
        </td>
        </tr>
    """
    tbl += """
    </table>
    </tbody>
    """

    ns = ns.replace('{%ping_table%}',tbl)

    segments += ns

template = template.replace('{%segments%}',segments)

file = os.path.join(DIR,'docs','index.html')
with open(file,'w',encoding='utf8') as f:
    f.write(template)